In [219]:
from patbert.visualization import embeddings
import importlib
importlib.reload(embeddings)

<module 'patbert.visualization.embeddings' from 'c:\\users\\fjn197\\phd\\projects\\pipelines\\patbert\\patbert\\visualization\\embeddings.py'>

In [229]:
vector_nums = [7, 13, 8, 7]
vector_lens = [1.4, 1/4, 1/40, 1/250]
marker_sizes = [2*80, 2*10, .5, .01]
kwargs = {'seed':2, 'fontsize':19, 'zoom_sub_vec_num':9, 'dpi':100, 'text_x_shift':.1, 'text_y_shift':0, 'figsize':(16,10),
            'marker_multiplier':20}
embfig = embeddings.EmbeddingFigure(vector_nums, vector_lens, marker_sizes,**kwargs,
                save_path='..\\reports\\figures\\initial_embedding.png',
                add_inset=False, style='dark_background')
fig, ax, axins = embfig()
# turn off axes label and ticks
ax = embfig.turn_off_frame()
# Create the inset axis and plot the zoomed-in data
embfig.save_fig()

In [230]:
axins_kwargs = {'zoom':20, 'loc':7, 'x':-1, 'y':-1,
        'xlim':(-1.015,-.965), 'ylim':(-.79, -.72),
        'borderpad':0,
        'bbox_to_anchor':(.8, .5), 'mark_inset_loc':(4,2)}
embfig = embeddings.EmbeddingFigure(vector_nums, vector_lens, marker_sizes, **kwargs,
                axins_kwargs=axins_kwargs, 
                save_path='..\\reports\\figures\\initial_embedding_zoom.png',
                style='dark_background')
fig, ax, axins = embfig()
axins.text(0.25, .35, 'Neoplasms', transform=axins.transAxes, fontsize=21)
axins.text(0.78, 0.8, 'in situ', transform=axins.transAxes, fontsize=18)
axins.text(0.6, 0.65, 'Melanoma', transform=axins.transAxes, fontsize=12)
# turn off axes label and ticks
ax = embfig.turn_off_frame()
# Create the inset axis and plot the zoomed-in data
embfig.save_fig()


In [234]:
axins_kwargs = {'zoom':20, 'loc':7, 'x':-1, 'y':-1,
        'xlim':(-1.015,-.965), 'ylim':(-.79, -.72),
        'borderpad':0,
        'bbox_to_anchor':(.8, .5), 'mark_inset_loc':(4,2)}
kwargs['marker_multiplier'] = 30
kwargs['zoom_sub_vec_num'] = 9
kwargs['figsize'] = (10,10)
marker_sizes = [80, 10, 2, .3]
embfig = embeddings.EmbeddingFigure(vector_nums, vector_lens, marker_sizes, **kwargs,
                axins_kwargs=axins_kwargs, 
                save_path='..\\reports\\figures\\initial_embedding_zoom_light.png',
                style='seaborn-paper')
fig, ax, axins = embfig()
axins.text(0.25,.35, 'Neoplasms', transform=axins.transAxes, fontsize=21)
axins.text(0.7,0.8, 'in situ', transform=axins.transAxes, fontsize=18)
axins.text(0.5,0.65, 'Melanoma', transform=axins.transAxes, fontsize=12)
# turn off axes label and ticks
ax = embfig.turn_off_frame()
# Create the inset axis and plot the zoomed-in data
embfig.save_fig()